In [1]:
import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import datetime

import matplotlib.pyplot as plt

import data_loader

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

import importlib

import numpy as np
import pandas as pd
import scipy

import data_cache
import plot_tools
import psd_tool
from field_models import model

importlib.reload(plot_tools)
importlib.reload(data_loader)
importlib.reload(psd_tool)
importlib.reload(data_cache)

%matplotlib qt

In [2]:
start = datetime.datetime(year=2016, month=1, day=1)
end = datetime.datetime(year=2016, month=4, day=1)

In [3]:
l_cut = 5.0
tol = 0.25

In [4]:
refs_A = data_loader.load_compressed_rept_data(satellite="A", start=start, end=end)

fesa_A = refs_A["FESA"]
L_A = refs_A["L"]
mlt_A = refs_A["MLT"]
epoch_A = refs_A["EPOCH"]
energies = refs_A["ENERGIES"]

refs_B = data_loader.load_compressed_rept_data(satellite="B", start=start, end=end)

fesa_B = refs_B["FESA"]
L_B = refs_B["L"]
mlt_B = refs_B["MLT"]
epoch_B = refs_B["EPOCH"]
energies = refs_B["ENERGIES"]

satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))

fesa_A = fesa_A[:, (energies < 3.5)]
fesa_B = fesa_B[:, (energies < 3.5)]

fesa_A_DF = pd.DataFrame(fesa_A[satisfies_L_cut_A, :], index=epoch_A[satisfies_L_cut_A])
fesa_B_DF = pd.DataFrame(fesa_B[satisfies_L_cut_B, :], index=epoch_B[satisfies_L_cut_B])

fesa = pd.concat([fesa_A_DF, fesa_B_DF], sort=False).sort_index()
fesa = fesa.resample("5h").mean()

Loading REPT data between: 2016-01-01 00:00:00 and 2016-04-01 00:00:00.
Loading : REPT_201601_A.npz
Loading : REPT_201602_A.npz
Loading : REPT_201603_A.npz
Loading : REPT_201604_A.npz
Loading REPT data between: 2016-01-01 00:00:00 and 2016-04-01 00:00:00.
Loading : REPT_201601_B.npz
Loading : REPT_201602_B.npz
Loading : REPT_201603_B.npz
Loading : REPT_201604_B.npz


In [5]:
field_model = model.TS04D

B_l = 0.311653 / (l_cut**3)
E_l = [1.8, 2.1, 2.6, 3.4]
M_e = scipy.constants.physical_constants["electron mass energy equivalent in MeV"][0]

mu_vals = [(E**2 + 2 * M_e * E) / (2 * M_e * B_l) for E in E_l]
# mu_vals = [500, 1000, 3000, 5000, 8000]
chosen_k = 0.07

In [6]:
psd_refs_A = data_loader.load_psd(satellite="A", field_model=field_model, start=start, end=end)
psd_refs_B = data_loader.load_psd(satellite="B", field_model=field_model, start=start, end=end)

2016-01-01 00:00:00
Loading : PSD_201601_A_TS04D.npz
2016-02-01 00:00:00
Loading : PSD_201602_A_TS04D.npz
2016-03-01 00:00:00
Loading : PSD_201603_A_TS04D.npz
2016-04-01 00:00:00
Loading : PSD_201604_A_TS04D.npz
2016-01-01 00:00:00
Loading : PSD_201601_B_TS04D.npz
2016-02-01 00:00:00
Loading : PSD_201602_B_TS04D.npz
2016-03-01 00:00:00
Loading : PSD_201603_B_TS04D.npz
2016-04-01 00:00:00
Loading : PSD_201604_B_TS04D.npz


In [9]:
for chosen_mu in mu_vals:

    print(f"Selecting PSD for A at mu = {chosen_mu}, k = {chosen_k}")

    selected_psd_refs_A = psd_tool.select_mu_and_k_from_psd(
        refs=psd_refs_A, chosen_mu=chosen_mu, chosen_k=chosen_k
    )

    print(f"Selecting PSD for B at mu = {chosen_mu}, k = {chosen_k}")

    selected_psd_refs_B = psd_tool.select_mu_and_k_from_psd(
        refs=psd_refs_B, chosen_mu=chosen_mu, chosen_k=chosen_k
    )

    data_cache.cache_psd_at_selected_mu_and_k(
        selected_psd_refs_A,
        satellite="A",
        start=start,
        end=end,
        mu=chosen_mu,
        k=chosen_k,
        field_model=field_model,
    )
    data_cache.cache_psd_at_selected_mu_and_k(
        selected_psd_refs_B,
        satellite="B",
        start=start,
        end=end,
        mu=chosen_mu,
        k=chosen_k,
        field_model=field_model,
    )

Selecting PSD for A at mu = 1993.5075698637097, k = 0.07
Time taken for loop: 37.36885356903076
Selecting PSD for B at mu = 1993.5075698637097, k = 0.07
Time taken for loop: 30.733114004135132
Selecting PSD for A at mu = 2573.0048161529007, k = 0.07
Time taken for loop: 30.79384732246399
Selecting PSD for B at mu = 2573.0048161529007, k = 0.07
Time taken for loop: 30.68855333328247
Selecting PSD for A at mu = 3695.815137520753, k = 0.07
Time taken for loop: 31.117708921432495
Selecting PSD for B at mu = 3695.815137520753, k = 0.07
Time taken for loop: 31.00031590461731
Selecting PSD for A at mu = 5900.463753345901, k = 0.07
Time taken for loop: 31.08346462249756
Selecting PSD for B at mu = 5900.463753345901, k = 0.07
Time taken for loop: 30.705978631973267


In [10]:
fig, axs = plt.subplots(2, 1, sharex=True)


num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):

    axs[0].semilogy(
        fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5
    )

axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color="gray", linestyle="dashed")
axs[0].xaxis.grid(color="gray", linestyle="dashed")
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED, SPIN-AVERAGED")


num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):

    selected_psd_refs_A = data_cache.load_psd_cache(
        satellite="A", start=start, end=end, mu=chosen_mu, k=chosen_k, field_model=field_model
    )
    selected_psd_refs_B = data_cache.load_psd_cache(
        satellite="B", start=start, end=end, mu=chosen_mu, k=chosen_k, field_model=field_model
    )

    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"]
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]

    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]

    satisfies_Lstar_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_Lstar_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))

    epoch_A_cut = epoch_A[satisfies_Lstar_cut_A]
    PSD_A_cut = PSD_A[satisfies_Lstar_cut_A]

    epoch_B_cut = epoch_B[satisfies_Lstar_cut_B]
    PSD_B_cut = PSD_B[satisfies_Lstar_cut_B]

    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]

    axs[1].semilogy(
        PSD.index, PSD, linewidth=2, color=mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)"
    )


axs[1].yaxis.grid(color="gray", linestyle="dashed")
axs[1].xaxis.grid(color="gray", linestyle="dashed")
axs[1].set_title(
    rf"L* = {l_cut}, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}, RBSP-COMBINED"
)
axs[1].set_ylabel("PSD $(c/MeV/cm)^{3}$")
axs[1].legend(loc="upper left")
axs[1].set_xlim(start, end)

C:\Users\brofe\AppData\Local\Temp\ipykernel_31524\3504672947.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)


(15706.0, 15901.0)